In [1]:
from comparative_translator import Locale_Translation
from excel_tools import Ranked_Translations, methods, dict2excel
from online_translator import neuro_translate, translate_phrases
from utilities import dict2json, file2dict
from tqdm import tqdm


In [2]:
locale_path = "Locales_1_21"
loc_par = [
    ["en_US_1_21.lang", "en_us_1_21.json"],
    ["nb_NO_1_21.lang", "no_NO_1_21.json"],
    ["fi_FI_1_21.lang","fi_FI_1_21.json"]
]
target_loc = "se_NO_1_21.json"
out_gen_trans = "se_no_ranked.json"

en_lang = file2dict("Locales_1_21/en_US_1_21.lang")
en_json = file2dict("Locales_1_21/en_us_1_21.json")
sme_json = file2dict("Locales_1_21/se_NO_1_21.json")
sme_old_lang = file2dict("Locales_1_7/se_NO.lang")


In [3]:
old_lang_keys = set(sme_old_lang.keys())
en_lang_keys = set(en_lang.keys())

print(len(old_lang_keys), len(en_lang_keys))

shared_keys = old_lang_keys.intersection(en_lang_keys)

missing_keys = en_lang_keys - shared_keys

print(len(shared_keys))

8801 11141
7890


In [4]:
lt = Locale_Translation(loc_par, target_loc, LOCALE_PATH=locale_path)
lt.find_corresponding_keys()  # Use english dictionary keys as basis

string index out of range
Format error in lang file, Line 1:
﻿

string index out of range
Format error in lang file, Line 1:
﻿

Finding corresponding keys.


100%|██████████| 11141/11141 [02:09<00:00, 86.26it/s] 


In [5]:
for p in lt.pairs:
    for k in shared_keys:
        if k in p.corresponding.keys():
            del p.corresponding[k]
    print(len(p.corresponding.keys()))
# 765, 258



765
258
313


In [6]:
generated = lt.rank_and_generate()

In [7]:
for k in shared_keys:
    generated[k] = {
        "value": sme_old_lang[k],
        "rank": "FROM_OLD_LANG",
        "quality": 3
    }

In [9]:
#fin_lang = file2dict("Locales_1_21/fi_FI_1_21.lang")
#left_keys = set(fin_lang.keys()) - set(generated.keys())
nob_lang = file2dict("Locales_1_21/nb_NO_1_21.lang")
left_keys = set(nob_lang.keys()) - set(generated.keys())
"""
finsme = {}
for k in tqdm(left_keys):
    trans = neuro_translate(fin_lang[k], "fin", "sme")
    generated[k] = {
        "value": trans,
        "rank": "MACHINE_TRANSLATED",
        "quality": 0
    }
    finsme[k] = trans
"""
#phrases = [finsme[k] for k in left_keys]
phrases = [nob_lang[k] for k in left_keys]
print(len(en_lang_keys), len(shared_keys), len(phrases), len(en_lang_keys)-len(phrases)-len(shared_keys))

translated = await translate_phrases(phrases, "nor", "sme")
#translated = await translate_phrases(phrases, "fin", "sme")

nobsme = {}
for i, k in enumerate(left_keys):
    nobsme[k] = translated[i]
    generated[k] = {
        "value": translated[i],
        "rank": "MACHINE_TRANSLATED",
        "quality": 0
    }

string index out of range
Format error in lang file, Line 1:
﻿

11141 7890 2436 815


100%|██████████| 2436/2436 [10:35<00:00,  3.83it/s]


In [13]:
rt = Ranked_Translations(generated)
rt.to_excel("ranked_sme_from_nob.xlsx", "Locales_1_21/en_US_1_21.lang")

In [11]:
#dict2excel(finsme, "finsme.xlsx", "Locales_1_21/en_US_1_21.lang")
dict2excel(nobsme, "nobsme.xlsx", "Locales_1_21/en_US_1_21.lang")

In [12]:
dict2json(generated, "ranked.json")

NameError: name 'left_keys' is not defined